In [3]:
from pathlib import Path
import cv2
from tqdm.notebook import tqdm
import numpy as np
import argparse
import cv2
import imutils
import json

In [ ]:
coefs = {}

MASK_DIR_PATH = "../../data/colorchecker_masks/"
EPSILON = 0.0001 # as much epsilon as rough colorchecker approximation
REAL_AREA = 63.5 * 108

In [4]:
for file in tqdm([*(Path(MASK_DIR_PATH)).glob('*.jpg')]):
    img = cv2.imread(str(file))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    contours = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    longest_contour = max(contours, key=cv2.contourArea)

    perimeter = cv2.arcLength(longest_contour, True)
    approximation = cv2.approxPolyDP(longest_contour, EPSILON * perimeter, True)

    empty = np.zeros((img.shape[0], img.shape[1], 1), dtype=np.uint8)

    cv2.fillPoly(empty, [approximation], 255)

    cv2.imwrite("../../data/approxed_colorchecker/" + str(file.stem) + ".png", empty)

    whitePixelCount = cv2.countNonZero(img)
    area_ratio = whitePixelCount / REAL_AREA

    coefs[str(file.stem)] = area_ratio

  0%|          | 0/1701 [00:00<?, ?it/s]

In [5]:
with open('coefs.json', 'w') as f:
    json.dump(coefs, f)